Python code at https://github.com/Haikson/gbscrap/blob/master/unit1/task3.py


Ваше задание:

Источник: https://geekbrains.ru/posts

Задача:

необходимо обойти все публикации в блоге и сохранить в MongoDB следующую информацию: Заголовок статьи, ссылка на статью, дата публикации (только дата без времени), Имя автора

Обход делаем с помощью BS4 + Requests

In [30]:
import requests
from bs4 import BeautifulSoup

In [31]:
def get_page_content(url):
    resp = requests.get(url)
    if resp.status_code >= 400:
        return ""
    else:
        return resp.text
    

In [32]:
base_url = "https://geekbrains.ru"
start_url = "https://geekbrains.ru/posts"
html = get_page_content(start_url)
soup = BeautifulSoup(html)

In [33]:
paginator = soup.findAll('ul', {'class': 'gb__pagination'})[0]
paginator.contents[-2].contents[0].text

'53'

In [34]:
def norm_link(url: str):
    if url.startswith('/'):
        return base_url + url
    return url

In [35]:
def get_articles_list(html):
    soup = BeautifulSoup(html)
    links = set()
    for link in soup.findAll('a', {'class': 'post-item__title'}):
        links.add(norm_link(link['href']))
    return links
    
def get_all_articles_list_pages():
    soup = BeautifulSoup(get_page_content(start_url))
    paginator = soup.find('ul', {'class': 'gb__pagination'})
    pages_count = int(paginator.contents[-2].contents[0].text)
    links = set()
    for i in range(1, pages_count+1):
        url = start_url + '?page={}'.format(i)
        links.update(list(get_articles_list(get_page_content(url))))
    return links

In [36]:
urls_todo = get_all_articles_list_pages()

In [37]:
from pymongo import MongoClient
mongo_client = MongoClient('localhost', 27017)

In [38]:
db = mongo_client['gb_scrap']
collection = db.gb_posts

In [39]:
collection.deleteMany()

['gb_posts']

In [40]:
def add_to_mongo(collection, title, url, date, author):
    """
    Заголовок статьи
    ссылка на статью
    дата публикации (только дата без времени)
    Имя автора
    :return: 
    """
    collection.insert({
        'title': title,
        'url': url,
        'date': date,
        'author': author
    })

In [41]:
from datetime import datetime
def parse_article(url):
    html = get_page_content(url)
    soup = BeautifulSoup(html)
    title = soup.find('h1', {'class': 'blogpost-title'}).text
    date = datetime.strptime(
        soup.find('time')['datetime'].split("T")[0],
        "%Y-%m-%d"
    )
    author = soup.find('div', {"itemprop": "author"}).text
    return {
        'title': title,
        'url': url,
        'date': date,
        'author': author
    }

In [ ]:
for url in urls_todo:
    try:
        add_to_mongo(collection=collection, **parse_article(url))
    except Exception as e:
        print(e)

/Users/k.petrosyan/dev/env/gbscrap/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  del sys.path[0]


In [ ]:
collection.count() == len(urls_todo)